## Waveform Processing

The .SAC files will be then further processed using "Bartlett" beamforming method and Adaptive F-Detector to identify signals most related to Anak Krakatau Volcanic Activity. These specific volcanic activity signals will be later ingested into the algorithm for the Deep Learning processing. This notebook will do as follows:
1. Implementing FK Beamforming to each station group
2. Running Adaptive F-Detector to Isolate relevant Waveform


In [ ]:
import sys
sys.path.append('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/')

import obspy
from obspy import UTCDateTime, read, read_inventory
from obspy.clients.fdsn.mass_downloader import GlobalDomain, \
    Restrictions, MassDownloader
from obspy.core.util.attribdict import AttribDict
import os
import subprocess
import glob
from tqdm.notebook import tqdm
from tqdm import tqdm
from src.utils.converter import *
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

### 1. FK Beamforming

In [ ]:
# 4.1 Running Beamforming on I06AU Waveform

base_folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_SAC'
df_grouped_file_paths = sac_path_grouping(base_folder_path)

# Wrap the iterrows with tqdm to create a progress bar
for index, row in tqdm(df_grouped_file_paths.iterrows(), total=df_grouped_file_paths.shape[0], desc="Processing"):
    # Construct the full path to the grouped files
    full_grouped_path = f"{base_folder_path}/{row['GroupedFilePath']}"

    # Prepare the CLI command
    cli_command = f"infrapy run_fk --local-wvfrms \"{full_grouped_path}\" --"
    
    try:
        # Execute the CLI command
        subprocess.run(cli_command, shell=True, check=True)
        print(f"Command executed for: {full_grouped_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the command for {full_grouped_path}: {e}")

In [ ]:
# 4.1 Running Beamforming on I52GB Waveform

base_folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I52GB/WAVEFORM_I52GB_SAC'
df_grouped_file_paths = sac_path_grouping(base_folder_path)

# Wrap the iterrows with tqdm to create a progress bar
for index, row in tqdm(df_grouped_file_paths.iterrows(), total=df_grouped_file_paths.shape[0], desc="Processing"):
    # Construct the full path to the grouped files
    full_grouped_path = f"{base_folder_path}/{row['GroupedFilePath']}"

    # Prepare the CLI command
    cli_command = f"infrapy run_fk --local-wvfrms \"{full_grouped_path}\" --"
    
    try:
        # Execute the CLI command
        subprocess.run(cli_command, shell=True, check=True)
        print(f"Command executed for: {full_grouped_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the command for {full_grouped_path}: {e}")

### 2. Running Adaptive F-Detector

In [ ]:
# 5.2 Running AFD for I06AU Station

folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FK_RESULTS'

# List all files in the folder
files = os.listdir(folder_path)
full_paths = [os.path.join(folder_path, file) for file in files]

# Using ThreadPoolExecutor to process files concurrently
with ThreadPoolExecutor(8) as executor:
    # Map the executor to the process_file function and the list of file paths
    results = list(tqdm(executor.map(lambda file_path: subprocess.run(f"infrapy run_fd --local-fk-label {file_path} --p-value 0.05 --back-az-width 5", shell=True), full_paths), total=len(full_paths), desc="Processing files"))


In [ ]:
# Path to the folder containing the files
folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I52GB/I52GB_FK'

# List all files in the folder
files = os.listdir(folder_path)
full_paths = [os.path.join(folder_path, file) for file in files]

# Using ThreadPoolExecutor to process files concurrently
with ThreadPoolExecutor(8) as executor:
    # Map the executor to the process_file function and the list of file paths
    results = list(tqdm(executor.map(lambda file_path: subprocess.run(f"infrapy run_fd --local-fk-label {file_path} --p-value 0.05 --back-az-width 5", shell=True), full_paths), total=len(full_paths), desc="Processing files"))


### 3. Running Spectral Yield Estrimation for Characterization

In [21]:
!infrapy run_spye single-station --local-wvfrms '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_SAC/IM.I06H*..BDF__20190226T000000Z__20190227T000000Z.sac' --local-detect-label '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FD_RESULTS/IM.I06H_2018.12.27_00.00.00-00.00.19.dets.json' --src-lat -6.102 --src-lon 105.423 --freq-min 0.7 --freq-max 4 --tlm-label '/run/media/viblab/Markov11/Haykal/infrapy/infrapy/propagation/priors/tloss/2007_08-' --local-pdf-label 'I06H'


########################################
##                                    ##
##              InfraPy               ##
##  Spectral Yield Estimation (SpYE)  ##
##       Single Station Analysis      ##
##                                    ##
########################################


Data parameters:
  local_detect_label: /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FD_RESULTS/IM.I06H_2018.12.27_00.00.00-00.00.19.dets.json
  det_index: 0
  tlm_label: /run/media/viblab/Markov11/Haykal/infrapy/infrapy/propagation/priors/tloss/2007_08-
  local_pdf_label: I06H
  local_loc_label: None
    src_lat: -6.102
    src_lon: 105.423
  local_wvfrms: /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_SAC/IM.I06H*..BDF__20190226T000000Z__20190227T000000Z.sac

Algorithm parameters:
  freq_min: 0.7
  freq_max: 4.0
  ref_rng: 1.0
  resolution: 200
  noise_option: post
  window_buffer: 0.2

Loading local data from /run/media/viblab/Markov2/Haykal/AnakKrakat

In [19]:
!infrapy run_spye single-station -h

Usage: infrapy run_spye single-station [OPTIONS]

  Run Spectral Yield Estimation (SpYE) methods to estimate the near-source
  acoustic spectral amplitude for a single detecting station

  Example usage (run from infrapy/examples directory):
      infrapy run_spye single-station --local-wvfrms '../infrapy-data/hrr-5/W220/*.sac' --local-detect-label data/HRR-5.dets.json --det-index 0 --src-lat 33.5377 --src-lon -106.333961 --tlm-label "../infrapy/propagation/priors/tloss/2007_08-" --local-pdf-label "HRR-5_W220"
      infrapy run_spye single-station --local-wvfrms '../infrapy-data/hrr-5/W240/*.sac' --local-detect-label data/HRR-5.dets.json --det-index 1 --src-lat 33.5377 --src-lon -106.333961 --tlm-label "../infrapy/propagation/priors/tloss/2007_08-" --local-pdf-label "HRR-5_W240"
      infrapy run_spye single-station --local-wvfrms '../infrapy-data/hrr-5/W340/*.sac' --local-detect-label data/HRR-5.dets.json --det-index 2 --src-lat 33.5377 --src-lon -106.333961 --tlm-label "../infrapy/pr

In [15]:
!infrapy run_spye regional --local-wvfrms '/run/media/viblab/Markov11/Haykal/infrapy/infrapy-data/hrr-5/*/*.sac' --local-detect-label '/run/media/viblab/Markov11/Haykal/infrapy/examples/data/HRR-5.dets.json' --tlm-label '/run/media/viblab/Markov11/Haykal/infrapy/infrapy/propagation/priors/tloss/2007_08-' --src-lat 33.5377 --src-lon -106.333961 --freq-min 0.25 --freq-max 2.0


########################################
##                                    ##
##              InfraPy               ##
##  Spectral Yield Estimation (SpYE)  ##
##   Regional (Single TLM) Analysis   ##
##                                    ##
########################################


Data parameters:
  local_detect_label: /run/media/viblab/Markov11/Haykal/infrapy/examples/data/HRR-5.dets.json
  tlm_label: /run/media/viblab/Markov11/Haykal/infrapy/infrapy/propagation/priors/tloss/2007_08-
  local_loc_label: None
    src_lat: 33.5377
    src_lon: -106.333961
  local_wvfrms: /run/media/viblab/Markov11/Haykal/infrapy/infrapy-data/hrr-5/*/*.sac

Algorithm parameters:
  freq_min: 0.25
  freq_max: 2.0
  yld_min: 1.0
  yld_max: 1000.0
  ref_rng: 1.0
  resolution: 200
  noise_option: post
  window_buffer: 0.2
  amb_press: 101.325
  amb_temp: 288.15
  grnd_burst: True
  exp_type: chemical

Loading local data from /run/media/viblab/Markov11/Haykal/infrapy/infrapy-data/hrr-5/*/*.sac

Detectio

In [12]:
!infrapy run_spye --help

Usage: infrapy run_spye [OPTIONS] COMMAND [ARGS]...

  infrapy run_spye - explosive yield estimation methods

Options:
  -h, --help  Show this message and exit.

Commands:
  combine         Combine near-source spectral amplitude PDFs from stations
  regional        Run analysis using a single set of TLMs
  single-station  Estimate the near-source spectral amplitude from a single
                  station


UnpicklingError: invalid load key, '#'.